In [41]:
from pydrake.all import (
    ModelVisualizer,
    StartMeshcat,
    PackageMap,
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    RigidTransform,
    Parser,
    RollPitchYaw,
    AddDefaultVisualization,
    Simulator,
)
import pydot

from IPython.display import HTML, SVG, display
from manipulation import running_as_notebook, ConfigureParser
import numpy as np

In [3]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.parser().package_map().AddRemote(
    package_name="spot_description",
    params=PackageMap.RemoteParams(
        urls=[
            f"https://github.com/bdaiinstitute/spot_ros2/archive/d429947a1df842ec38f8c6099dde9501945090d6.tar.gz"
        ],
        sha256=("e4dd471be4e7e822a12afcfd6a94ce7ecbb39e2d4ea406779a96e146a607bf53"),
        strip_prefix="spot_ros2-d429947a1df842ec38f8c6099dde9501945090d6/spot_description/",
    ),
)
visualizer.AddModels(url="package://manipulation/spot/spot_with_arm.urdf")
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

In [111]:
platform_sdf = """<?xml version="1.0"?>
    <sdf version="1.7">
    <model name="platform">
        <link name="platform_link">
        <visual name="visual">
            <pose>0 0 -0.2 0 0 0</pose>
            <geometry>
            <box>
                <size>10.0 8.1 0.05</size>
            </box>
            </geometry>
            <material>
            <diffuse>0.9 0.8 0.7 1.0</diffuse>
            </material>
        </visual>
        <collision name="collision">
            <pose>0 0 -0.2  0 0 0</pose>
            <geometry>
            <box>
                <size>10.0 8.1 0.05</size>
            </box>
            </geometry>
        </collision>
        </link>
        <frame name="platform_center">
        <pose relative_to="platform_link">0 0 0.47 0 0 0</pose>
        </frame>
    </model>
    </sdf>
    """

In [112]:
def create_scene(sim_time_step):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    parser = Parser(plant)
    # Add the manipulation/package.xml index to the given Parser, so it know
    # where to find Spot model
    ConfigureParser(parser)

    # Loading models.
    parser.package_map().AddRemote(
        package_name="spot_description",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/bdaiinstitute/spot_ros2/archive/d429947a1df842ec38f8c6099dde9501945090d6.tar.gz"
            ],
            sha256=("e4dd471be4e7e822a12afcfd6a94ce7ecbb39e2d4ea406779a96e146a607bf53"),
            strip_prefix="spot_ros2-d429947a1df842ec38f8c6099dde9501945090d6/spot_description/",
        ),
    )
    parser.AddModels(url="package://manipulation/spot/spot_with_arm.urdf")
    parser.AddModelsFromString(platform_sdf, "sdf")

    # # Weld the table to the world so that it's fixed during the simulation.
    platform_frame = plant.GetFrameByName("platform_center")
    plant.WeldFrames(plant.world_frame(), platform_frame)
    # # Finalize the plant after loading the scene.
    plant.Finalize()

    # Add visualization to see the geometries.
    AddDefaultVisualization(builder=builder, meshcat=meshcat)

    diagram = builder.Build()
    return diagram

In [113]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.0)
    return simulator


def run_simulation(sim_time_step):
    diagram = create_scene(sim_time_step)
    simulator = initialize_simulation(diagram)
    meshcat.StartRecording()
    finish_time = 2.0
    simulator.AdvanceTo(finish_time)
    meshcat.PublishRecording()

In [114]:
run_simulation(0.0001)

In [ ]:
diagram = create_scene(0.001)
display(
    SVG(
        pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[
            0
        ].create_svg()
    )
)